## UAP Generate & Evaluate (whitebox)

attacks
- sgd-uap (untargeted) eps = 2-12

dataset
- cifar10
- svhn

model_name
- resnet18
- [soft-pruning]
    - resnet18_sfP
    - resnet18_sfP-mixup
    - resnet18_sfP-cutout
- [post-training pruning]
    - resnet18_PX_0.Y
        - X = 2, 3, 4
        - Y = 3, 6, 9

In [1]:
import numpy as np
import pandas as pd
import os
import torch
import json

from utils_attack import uap_batch
from utils_model import *

dir_results = './results/'
dir_uap = './uaps/'

In [2]:
nb_epoch = 10
atk_name = 'sgd-uap'
y_target = -1

In [3]:
model_names =  ['resnet18', 'resnet18_sfP', 'resnet18_sfP-mixup', 'resnet18_sfP-cutout']
for layer in [2, 3, 4]:
    for prune_pct in ['0.3', '0.6', '0.9']:
        model_names.append('resnet18_P%i-%s' % (layer, prune_pct))
model_names.pop(6)
model_names.pop(8)
model_names

['resnet18',
 'resnet18_sfP',
 'resnet18_sfP-mixup',
 'resnet18_sfP-cutout',
 'resnet18_P2-0.3',
 'resnet18_P2-0.6',
 'resnet18_P3-0.3',
 'resnet18_P3-0.6',
 'resnet18_P4-0.3',
 'resnet18_P4-0.6',
 'resnet18_P4-0.9']

In [4]:
atk_sources = model_names.copy()
for Q_param in [2, 3, 4]:
    atk_sources.append('resnet20_Q%i' % Q_param)
atk_sources

['resnet18',
 'resnet18_sfP',
 'resnet18_sfP-mixup',
 'resnet18_sfP-cutout',
 'resnet18_P2-0.3',
 'resnet18_P2-0.6',
 'resnet18_P3-0.3',
 'resnet18_P3-0.6',
 'resnet18_P4-0.3',
 'resnet18_P4-0.6',
 'resnet18_P4-0.9',
 'resnet20_Q2',
 'resnet20_Q3',
 'resnet20_Q4']

In [5]:
def eval_uap(uap):    
    top, top_probs, top1acc, top5acc, outputs, labels = evaluate(model, testloader, uap = uap)

    print('Top 1 accuracy', sum(top1acc) / len(labels))
    print('Top 5 accuracy', sum(top5acc) / len(labels))

    dict_entry = {}
    dict_entry['model_name'] = model_name
    dict_entry['dataset'] = dataset
    
    dict_entry['atk_source'] = atk_source
    dict_entry['atk_name'] = atk_name
    dict_entry['atk_param'] = y_target
    dict_entry['eps'] = eps
    if atk_name[:3] == 'sgd': dict_entry['nb_epoch'] = nb_epoch

    dict_entry['top1acc'] = sum(top1acc) / len(labels)
    dict_entry['top5acc'] = sum(top5acc) / len(labels)
    dict_entry['UER'] = 1 - sum(top1acc) / len(labels)
    print('UER ', dict_entry['UER'])
    print('\n')
    if y_target >= 0: dict_entry['tgt_success'] = sum(outputs == y_target) / len(labels)

    # Output distribution
    for i in range(10):
        dict_entry['label_dist%i' % i] = sum(outputs == i) / len(labels)

    all_results.append(dict_entry)

In [6]:
Q_param = 3

In [7]:
for dataset in ['cifar10', 'svhn']:
    all_results = []
    
    testloader = get_testdata(dataset)

    for model_name in ['resnet20_Q%i' % Q_param]:
        model = get_model(model_name, dataset)
        model.eval()

        for atk_source in atk_sources:
            uap_load_path = dir_uap + dataset + '/' + atk_source
            
            for eps in range(2, 13, 2):
                uap_path = uap_load_path + '/sgd-eps%i.pth' % eps
                print(uap_path)
                uap = torch.load(uap_path)
                eval_uap(uap)

            pd.DataFrame(all_results).to_csv(dir_results + 'evalQ%i_untgt-transfer-%s.csv' % (Q_param, dataset))

    pd.DataFrame(all_results).to_csv(dir_results + 'evalQ%i_untgt-transfer-%s.csv' % (Q_param, dataset))

Files already downloaded and verified
Loading cifar10/resnet20_Q3.pth
./uaps/cifar10/resnet18/sgd-eps2.pth
 Completed: 0.00%
Top 1 accuracy 0.9049
Top 5 accuracy 0.9965
UER  0.09509999999999996


./uaps/cifar10/resnet18/sgd-eps4.pth
 Completed: 0.00%
Top 1 accuracy 0.8042
Top 5 accuracy 0.9863
UER  0.19579999999999997


./uaps/cifar10/resnet18/sgd-eps6.pth
 Completed: 0.00%
Top 1 accuracy 0.6794
Top 5 accuracy 0.982
UER  0.3206


./uaps/cifar10/resnet18/sgd-eps8.pth
 Completed: 0.00%
Top 1 accuracy 0.4709
Top 5 accuracy 0.954
UER  0.5291


./uaps/cifar10/resnet18/sgd-eps10.pth
 Completed: 0.00%
Top 1 accuracy 0.3261
Top 5 accuracy 0.9043
UER  0.6738999999999999


./uaps/cifar10/resnet18/sgd-eps12.pth
 Completed: 0.00%
Top 1 accuracy 0.1516
Top 5 accuracy 0.848
UER  0.8484


./uaps/cifar10/resnet18_sfP/sgd-eps2.pth
 Completed: 0.00%
Top 1 accuracy 0.9169
Top 5 accuracy 0.9964
UER  0.08309999999999995


./uaps/cifar10/resnet18_sfP/sgd-eps4.pth
 Completed: 0.00%
Top 1 accuracy 0.8193
Top 

 Completed: 0.00%
Top 1 accuracy 0.1779
Top 5 accuracy 0.8726
UER  0.8221


./uaps/cifar10/resnet20_Q2/sgd-eps2.pth
 Completed: 0.00%
Top 1 accuracy 0.9019
Top 5 accuracy 0.9969
UER  0.09809999999999997


./uaps/cifar10/resnet20_Q2/sgd-eps4.pth
 Completed: 0.00%
Top 1 accuracy 0.8323
Top 5 accuracy 0.9903
UER  0.16769999999999996


./uaps/cifar10/resnet20_Q2/sgd-eps6.pth
 Completed: 0.00%
Top 1 accuracy 0.6978
Top 5 accuracy 0.9756
UER  0.3022


./uaps/cifar10/resnet20_Q2/sgd-eps8.pth
 Completed: 0.00%
Top 1 accuracy 0.5519
Top 5 accuracy 0.9309
UER  0.44810000000000005


./uaps/cifar10/resnet20_Q2/sgd-eps10.pth
 Completed: 0.00%
Top 1 accuracy 0.4319
Top 5 accuracy 0.8553
UER  0.5681


./uaps/cifar10/resnet20_Q2/sgd-eps12.pth
 Completed: 0.00%
Top 1 accuracy 0.3667
Top 5 accuracy 0.8271
UER  0.6333


./uaps/cifar10/resnet20_Q3/sgd-eps2.pth
 Completed: 0.00%
Top 1 accuracy 0.9052
Top 5 accuracy 0.9964
UER  0.0948


./uaps/cifar10/resnet20_Q3/sgd-eps4.pth
 Completed: 0.00%
Top 1 accurac

./uaps/svhn/resnet18_P3-0.3/sgd-eps6.pth
 Completed: 0.00%
Top 1 accuracy 0.7614474492931776
Top 5 accuracy 0.9668869084204057
UER  0.23855255070682235


./uaps/svhn/resnet18_P3-0.3/sgd-eps8.pth
 Completed: 0.00%
Top 1 accuracy 0.6057160417947142
Top 5 accuracy 0.9211355255070682
UER  0.39428395820528583


./uaps/svhn/resnet18_P3-0.3/sgd-eps10.pth
 Completed: 0.00%
Top 1 accuracy 0.5562768899815611
Top 5 accuracy 0.9045789797172711
UER  0.4437231100184389


./uaps/svhn/resnet18_P3-0.3/sgd-eps12.pth
 Completed: 0.00%
Top 1 accuracy 0.4242470805162876
Top 5 accuracy 0.8165719114935464
UER  0.5757529194837123


./uaps/svhn/resnet18_P3-0.6/sgd-eps2.pth
 Completed: 0.00%
Top 1 accuracy 0.9444145666871543
Top 5 accuracy 0.9938153042409342
UER  0.05558543331284571


./uaps/svhn/resnet18_P3-0.6/sgd-eps4.pth
 Completed: 0.00%
Top 1 accuracy 0.8873309772587584
Top 5 accuracy 0.9885525507068224
UER  0.11266902274124158


./uaps/svhn/resnet18_P3-0.6/sgd-eps6.pth
 Completed: 0.00%
Top 1 accuracy 0.